In [33]:
import numpy as np
class MDP:
    def __init__(self, nb_questions=4, win_proba=[9/10, 3/4, 1/2, 1/10], reward=[100,1000,10000,50000]):
        self.nb_questions = nb_questions
        self.play_probs = win_proba
        self.rewards = reward
        self.q_table = np.zeros((nb_questions, 2))
        self.reward_table = np.zeros((nb_questions, 2))


    def call_action(self, question):
        return self.rewards[question]*np.random.binomial(1, self.play_probs[question], 1)

    def bellman_op(self, gamma=0.99):
        cum_sum = 0
        for i in range(self.nb_questions):  # Evaluate cumulative rewards per action and state
            for j in range(2):
                self.reward_table[i,j] = (1-j)*cum_sum + j*(cum_sum+self.rewards[i])
                cum_sum += j*self.rewards[i]

        for i in range(self.nb_questions-1, -1, -1):    # Compute
            for j in range(2):
                if i==self.nb_questions-1:
                    self.q_table[i,j] = (1-j)*(self.reward_table[i,j]) + j*(self.play_probs[i]*self.reward_table[i,j])
                else:
                    self.q_table[i,j] = (
                        (1-j)*(self.reward_table[i,j])
                        + j*(self.play_probs[i]*gamma*np.max(self.q_table[i+1]))
                    )


class Agent:
    def __init__(self, environment):
        self.question = 0   # The rank of the question
        self.won = False
        self.lost = False
        self.summed_reward = 0
        self.env = environment
        print("Welcome to the game! You are at question 1. Do you play?")

    def play(self):
        if not self.lost:
            immediate_reward = self.env.call_action(self.question)
            self.summed_reward += immediate_reward
            if immediate_reward == 0:
                self.lost = True
                print("You lost :( \n   You exit the game with a gain of {}".format(self.summed_reward))
            elif self.question==self.env.nb_questions:
                print("Congratulations, you won!! :) \n   Your total gains amount to {}".format(self.summed_reward))
            else:
                self.question += 1
                print("You passed! You are now at question {}. \n   Your total gains are {}".format(self.question+1, self.summed_reward))
        else:
            print("Bro the game is over... Take your {} euros and go home now.".format(self.summed_reward))

    def quit(self):
        if not self.won:
            self.lost = True


In [36]:
environment = MDP()
environment.bellman_op()
print(environment.q_table)
print(environment.reward_table)

[[    0.          3634.98262875]
 [  100.          4079.66625   ]
 [ 1100.          5494.5       ]
 [11100.          6110.        ]]
[[    0.   100.]
 [  100.  1100.]
 [ 1100. 11100.]
 [11100. 61100.]]
